# Library Imports

In [1]:
import librosa
import os
import pickle
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import DataLoader, TensorDataset
from torchaudio.prototype.pipelines import VGGISH
import openl3

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

device = "cuda" if torch.cuda.is_available() else "cpu"

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# MVP 1

In [2]:
def load_and_split(audios, sr, segments=3):
    """
    Loads in the mp3 files available for the model and splits each into x segments. Default is 3 so each segment will be 5 seconds long (15 second total each)

    Args:
        audios -> Dictionary of audio ids and full audio path
                    e.g. {0:'...7horsemethlabzosostickerthewolfofwallstreetsample1mp3.mp3'}
        sr -> Desired sample rate for loading in the audio files with
                Audio files are natively 44.1k but not set as default to enable use in other functions

    Outputs:
        1. Audio IDs -> Numpy array, duplicates as ID will be copied in for each segment (use for finding the corresponding film metadata)
        2. Librosa loaded audio samples -> 2D Numpy array, each row corresponds to a unique audio segment
    """

    ids, data = [], []

    for id, path in tqdm(audios.items(), desc="Processing audio files"):
        signal, _ = librosa.load(path, sr=sr)
        segment_length = len(signal) / segments

        # Librosa loaded file is just a numpy array so possible to segment using slicing
        for i in range(segments):
            start = int(i * segment_length)
            end = int(start + segment_length)
            samples = signal[start:end]
            ids.append(id), data.append(samples)

    return np.array(ids), np.array(data)

def feature_extraction(data, sr, hop_length=1024, n_fft=4096):
    """
    Extracts audio features from the librosa time series and standardises

    Args:
        data -> Numpy array of Librosa loaded audio segments
        sr -> Sample rate
        hop_length & n_fft -> Hyperparameters to be shared across various features

    Output:
        X -> Numpy matrix of audio features of shape (# samples, # features)
    """

    X = []

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for i in tqdm(range(len(data)), desc="Extracting audio features"):
            y = data[i]
            
            # --- Timbral Features ---
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, n_mels=60, n_fft=n_fft, hop_length=hop_length, fmin=20, fmax=22050, lifter=30, win_length=n_fft, window='hann')
            f_mfccmean = np.mean(mfccs, axis=1)
            f_mfccstd = np.std(mfccs, axis=1)
            f_speccentoid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft))
            f_specband = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft))
            f_speccontrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft))

            # --- Rhythm Features ---
            f_tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr, hop_length=hop_length)
            beat_times = librosa.frames_to_time(beat_frames, sr=sr, hop_length=hop_length, n_fft=n_fft)
            beat_indices = librosa.time_to_frames(beat_times, sr=sr, hop_length=hop_length, n_fft=n_fft)
            f_beatrms = np.nan_to_num(np.mean(librosa.feature.rms(y=y, hop_length=hop_length)[0, beat_indices])) # Error handling -> Convert to 0 as tempo will also be 0
            f_onsetstrength = np.mean(librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft))
            
            # --- Loudness Features ---
            rms = librosa.feature.rms(y=y, hop_length=hop_length)
            f_rmsmean = np.mean(rms)
            f_rmsstd = np.std(rms)

            # --- Harmonic Features ---
            chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
            f_chromamean = np.mean(chroma, axis=1)
            f_chromastd = np.std(chroma, axis=1) 

            features = np.hstack((f_mfccmean, f_mfccstd, f_speccentoid, f_specband, f_speccontrast, f_tempo, f_beatrms, f_onsetstrength, f_rmsmean, f_rmsstd, f_chromamean, f_chromastd))
            X.append(features)

    return np.array(X)

def feature_selection_standardised(X, kpca, standardise=True, var_threshold=0.15, corr_threshold=0.75, n_components=8):
    """
    Performs variance and correlation feature selection and kernel PCA on the feature matrix
    Drops columns with a variance less than var_threshold and drops one of two correlated features greater than corr_threshold
    Also standardises the data, very important for PCA

    Args:
        X -> Feature matrix (# samples, # features)
        kpca -> Boolean, performs PCA if true
        standardise -> Boolean, standardises feature matrix if true
        var_threshold -> Lower limit of acceptable variance, features with lower variance will be dropped
        corr_threshold -> Upper limit of acceptable correlation, only drops one of the two correlated features if correlation is greater than the threshold
        n_components -> Number of components for PCA

    Output:
        X_reduced -> Matrix with reduced features
    """

    X_reduced = VarianceThreshold(threshold=var_threshold).fit_transform(X)

    df = pd.DataFrame(X_reduced)
    corrs = df.corr().abs()         # Only absolute size of correlation matters, not direction

    # 1. Returns boolean matrix of same shape as correlation matrix. True for the values above the self correlation diagonal
    # 2. Transforms the correlation matrix -> All NaN below and including the diagonal, keeps values above the diagonal
    upper = corrs.where(np.triu(np.ones(corrs.shape), k=1).astype(bool))
    X_reduced = df.drop(columns=[col for col in upper.columns if any(upper[col] > corr_threshold)])     # Finds columns in correlation matrix with any correlation higher than the threshold
    
    if standardise: X_reduced = StandardScaler().fit_transform(X_reduced)

    if kpca:
        kpca = KernelPCA(n_components=n_components)
        X_reduced = kpca.fit_transform(X_reduced)
        for i, l in enumerate(kpca.eigenvalues_): print(f'Component {i} roughly explains {l / sum(kpca.eigenvalues_):.3%} variance')

    return X_reduced

def evaluate_plot(X, model, k=30):
    """
    Evaluates multiple unsupervised metrics against the number of clusters
    
    Args:
        X -> Standardised feature matrix
        model -> Model to use, either GMM or kMeans
        k -> Max clusters to test
    
    Outputs:
        Plots of metric v cluster
    """

    clusters, CH, SS, DB = [], [], [], []

    if model == 'kmeans':
        for k in range(2, k):
            kmeans = KMeans(n_clusters=k, random_state=42).fit(X)
            labels = kmeans.labels_
            clusters.append(k), CH.append(calinski_harabasz_score(X, labels)), SS.append(silhouette_score(X, labels)), DB.append(davies_bouldin_score(X, labels))

    elif model == 'gmm':
        for k in range(2, k):
            gmm = GaussianMixture(n_components=k, random_state=42).fit(X)
            labels = gmm.predict(X)
            clusters.append(k), CH.append(calinski_harabasz_score(X, labels)), SS.append(silhouette_score(X, labels)), DB.append(davies_bouldin_score(X, labels))

    metrics = [
        (CH, 'Calinski-Harabasz Score', 'Calinski-Harabasz v Number of Clusters'),
        (SS, 'Silhouette Score', 'Silhouette Score v Number of Clusters'),
        (DB, 'Davies-Bouldin Score', 'Davies-Bouldin v Number of Clusters')]

    fig, ax = plt.subplots(1, 3, figsize=(20, 6))
    for i, (metric, label, title) in enumerate(metrics):
        ax[i].plot(clusters, metric)
        ax[i].set_ylabel(label, fontweight='bold')
        ax[i].set_xlabel('Clusters', fontweight='bold')
        ax[i].set_title(title, fontsize=14)
    plt.show()
    
    return None

def cluster_viz(X, model, clusters):
    """
    Visualises cluster assignment for a feature matrix of just 2 features
    Use PCA to reduce feature matrix to 2 components

    Args:
        X -> Standardised feature matrix of 2 features
        model -> Clustering model to use, k-means or gmm
        clusters -> Number of clusters to use in model

    Output:
        Plot of colour-coded clusters
    """

    if model == 'kmeans':
        labels = KMeans(n_clusters=clusters, random_state=42).fit_predict(X)
    elif model == 'gmm':
        labels = GaussianMixture(n_components=clusters, random_state=42).fit_predict(X)

    fig, ax = plt.subplots(1, 1, figsize=(9, 6))
    scatter = ax.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis')
    ax.set_title(f'{model}\nk = {clusters}', fontweight='bold', fontsize=17)
    ax.legend(*scatter.legend_elements(), title='Clusters')
    plt.show()
    
    return None

def cluster_predict(X, clusters, sample_audio_ids, audio_dict, AV_scores):
    """
    Clusters the feature matrix using k-Means and aggregates samples from the same audio file together for each label.
    For example, assume 2 samples from audio ID 54 are labelled as cluster 3. The median of these two samples will be calculated and stored with the process repeated for each cluster

    Aggregates (median) arousal-valence scores for each unique audio file, given that participants scored the same audio file multiple times and multiple participants scored the same audio file
    Aggregates familiarity ratings according to the highest count

    Args:
        X -> Standardised feature matrix
        clusters -> Number of clusters to use in k-means
        sample_audio_ids -> List of audio IDs for each sample in the feature matrix
        audio_dict -> Dictionary of audio IDs to filepaths --- {0:'...7horsemethlabzosostickerthewolfofwallstreetsample1mp3.mp3'}
        AV_scores -> Dataframe of arousal-valence scores and film metadata

    Outputs:
        1. cluster_df -> Dataframe of filepaths, clusters, aggregate arousal + valence, and film metadata
        2. median_cluster_scores -> Dataframe of median arousal + valence for each cluster

    """

    # ----- Clustering + Retrieving distance to assigned cluster -----
    kmeans = KMeans(n_clusters=clusters, random_state=42).fit(X)
    dist, label = kmeans.transform(X), kmeans.predict(X)
    dist_assigned = dist[np.arange(X.shape[0]), label]

    # ----- Aggregate Distance for Samples from same Audio in same Cluster -----
    dist_df = pd.DataFrame({'file':sample_audio_ids, 'label':label, 'dist_centroid':dist_assigned})     \
                .groupby(['file','label'])['dist_centroid'].median().reset_index()                      \
                .sort_values(by=['label','dist_centroid']).reset_index(drop=True)
    dist_df['file'] = [os.path.basename(audio_dict[id]) for id in dist_df['file']]

    # ----- Aggregate Valence, Arousal, Familiarity + Join with AV_scores for Film Metadata -----
    agg_AV = AV_scores.groupby('file').agg({'valence':'median','arousal':'median','familiarity':lambda x: x.value_counts().index[0]})
    agg_AV = pd.merge(agg_AV, AV_scores.drop(columns=['familiarity','arousal','valence']).drop_duplicates(subset=['file']).set_index('file'), on='file').reset_index()

    # ----- Merge Distance with Aggregate AV + AV Aggregation for Clusters -----
    cluster_df = pd.merge(dist_df, agg_AV, how='left', on='file')
    median_cluster_scores = cluster_df.groupby('label').agg({'valence':'median', 'arousal':'median'}).reset_index()

    return cluster_df, median_cluster_scores

def find_recommendations(cluster_df, cluster_scores, top_k=5):
    """
    Requests an input from the user for an arousal-valence score
    Calculates Euclidean distances between input and AV scores of each of the centroids - Returns cluster label
    Uses cluster label to filter clustered samples for only the relevant cluster, sorts by distance to centroid and returns the top_k audio tracks with the shortest distances

    Args:
        cluster_df -> Dataframe of clustered audio tracks including their AV scores, distance to assigned centroid and label
        cluster_scores -> Median AV scores of all the audio tracks assigned to each cluster
        top_k -> k results to retrieve

    Output:
        retrieved_tracks -> Dataframe of audio tracks assigned to the closest centroid to the input Arousal-Valence score. Length of dataframe = top_k
    """

    # ----- Arousal Input -----
    while True:
        try:
            arousal = float(input('Please input an arousal score between -1 and +1'))
            if abs(arousal) > 1:
                print('Please enter a number between -1 and +1')
                continue
            break
        except:
            print('Please enter a valid number')
            continue

    # ----- Valence Input -----
    while True:
        try:
            valence = float(input('Please input an valence score between -1 and +1'))
            if abs(valence) > 1:
                print('Please enter a number between -1 and +1')
                continue
            break
        except:
            print('Please enter a valid number')
            continue

    
    input_score = np.array([arousal, valence])
    centroids = cluster_scores[['arousal','valence']].to_numpy()
    dist = np.linalg.norm(centroids - input_score, axis=1)              # Euclidean distance between input AV score and AV scores of each of the centroids
    closest_centroid = cluster_scores.iloc[np.argmin(dist)]['label']    # Finds index of minimum distance and uses this to filter cluster_scores
    
    retrieved_tracks = cluster_df.loc[cluster_df['label'] == closest_centroid].sort_values(by='dist_centroid', ignore_index=True).head(top_k)
    print(f'Arousal-Valence: ({str(input_score[0])}, {str(input_score[1])})')

    return retrieved_tracks

In [3]:
# AV_scores = pd.read_csv(os.path.join(os.getcwd(), '..', 'CSV_files', 'arousal_valence.csv'))
# audio_dict = {index:os.path.join(os.getcwd(), '..', 'audios', file) for index, file in enumerate(sorted(AV_scores['file'].unique()))}

# ids, data  = load_and_split(audio_dict, sr=44100)
# X = feature_extraction(data, sr=44100)

In [4]:
# X_processed = feature_selection_standardised(X, kpca=True, n_components=2)
# cluster_viz(X_processed, model='kmeans', clusters=7)

In [5]:
# X_processed = feature_selection_standardised(X, kpca=True, n_components=6)
# evaluate_plot(X_processed, 'kmeans')

In [6]:
# X_processed = feature_selection_standardised(X, kpca=True, n_components=6)
# cluster_df, cluster_scores = cluster_predict(X_processed, 7, ids, audio_dict, AV_scores)

# cluster_df.to_csv(os.path.join(os.getcwd(), '..', 'CSV_files', 'tracks_clustered.csv'), index=False)
# cluster_scores.to_csv(os.path.join(os.getcwd(), '..', 'CSV_files', 'cluster_scores.csv'), index=False)

In [7]:
# find_recommendations(cluster_df, cluster_scores)


# MVP 2

Clustering component:
1. Extract embeddings using pretrained models (VGGish and Openl3)
2. Feed embeddings into a Variational Deep Embedding -> Integrate a VAE and GMM for clustering

Auxiliary variable component:
Use cross-modal retrieval with contrastive learning (similar to CLIP)
1. Encode features and encode arousal-valence so that they're on the same space
2. Train the model to push similar features together and dissimilar features apart

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim):
        super(Encoder, self).__init__()

        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.LeakyReLU(0.2))
            layers.append(nn.Dropout(0.2))
            prev_dim = h_dim
        self.encoder = nn.Sequential(*layers)

        self.mu = nn.Linear(hidden_dims[-1], latent_dim)
        self.logvar = nn.Linear(hidden_dims[-1], latent_dim)

    def forward(self, X):
        X = self.encoder(X)
        mu, logvar = self.mu(X), self.logvar(X)
        std = torch.exp(0.5 * logvar)
        z = mu + std * torch.randn_like(std)
        return mu, logvar, z

class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim):
        super(Decoder, self).__init__()

        layers = []
        prev_dim = latent_dim
        for h_dim in hidden_dims[::-1]:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.LeakyReLU(0.2))
            layers.append(nn.Dropout(0.2))
            prev_dim = h_dim
        self.decoder = nn.Sequential(*layers)

        self.reconstructed = nn.Linear(hidden_dims[0], input_dim)

    def forward(self, z):
        z = self.decoder(z)
        return self.reconstructed(z)

class AVencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(AVencoder, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim, 64),
                                     nn.ReLU(),
                                     nn.Linear(64, output_dim))
        self.norm = nn.LayerNorm(output_dim)

    def forward(self, av):
        av = self.encoder(av)
        return self.norm(av)

class ProjectionHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ProjectionHead, self).__init__()
        self.projector = nn.Sequential(nn.Linear(input_dim, output_dim, bias=False),
                                       nn.BatchNorm1d(output_dim),
                                       nn.ReLU(),
                                       nn.Linear(output_dim, output_dim, bias=False),
                                       nn.BatchNorm1d(output_dim, affine=False))
    def forward(self, x):
        return self.projector(x)

class VaDE(nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, n_clusters):
        super(VaDE, self).__init__()
        self.input_dim, self.hidden_dims, self.latent_dim, self.n_clusters = input_dim, hidden_dims, latent_dim, n_clusters

        self.encoder = Encoder(input_dim, hidden_dims, latent_dim)
        self.decoder = Decoder(input_dim, hidden_dims, latent_dim)

        self.cluster_weights = nn.Parameter(torch.ones(n_clusters)/n_clusters)
        self.cluster_means = nn.Parameter(torch.randn(n_clusters, latent_dim) * 0.05)
        self.cluster_logvars = nn.Parameter(torch.randn(n_clusters, latent_dim) * 0.05 - 1)

        self.av_encoder = AVencoder(input_dim=2, output_dim=latent_dim)
        self.projector = ProjectionHead(input_dim=latent_dim, output_dim=latent_dim)

    def forward(self, X, AV):
        mu, logvar, z = self.encoder(X)
        recon_X = self.decoder(z)
        av_encoded = self.av_encoder(AV)
        return mu, logvar, z, recon_X, av_encoded
    
    def cluster_probabilities(self, z):
        cluster_weights = torch.softmax(self.cluster_weights, dim=0)
        cluster_logvars = self.cluster_logvars.clamp(min=-10, max=10)
        cluster_vars = torch.exp(cluster_logvars) + 1e-8

        z = z.unsqueeze(1).expand(-1, self.n_clusters, -1)

        log_2pi = np.log(2 * np.pi)
        squared_diff = (z - self.cluster_means).pow(2)
        log_prob_z_given_c = -0.5 * (torch.sum(cluster_logvars + squared_diff / cluster_vars, dim=2) + self.latent_dim * log_2pi)
        log_prob_c_given_z = torch.log(cluster_weights.unsqueeze(0) + 1e-8) + log_prob_z_given_c
        gamma = torch.softmax(log_prob_c_given_z, dim=1)
        return gamma

    def loss(self, X, recon_X, mu, logvar, gamma, current_epoch):
        recon_loss = F.mse_loss(recon_X, X, reduction='sum') / (X.size(0) + 1e-10)
        
        logvar = logvar.clamp(min=-15, max=15)
        cluster_logvars = self.cluster_logvars.clamp(min=-15, max=15)
        
        var = torch.exp(logvar) + 1e-10
        cluster_vars = torch.exp(cluster_logvars) + 1e-10
        
        mu = mu.unsqueeze(1)
        var = var.unsqueeze(1)
        
        kl_z_per_cluster = 0.5 * (
            cluster_logvars.unsqueeze(0) - logvar.unsqueeze(1) + 
            (var + (mu - self.cluster_means.unsqueeze(0)).pow(2)) / cluster_vars.unsqueeze(0) - 1
        )
        
        kl_z = torch.sum(gamma.unsqueeze(-1) * kl_z_per_cluster, dim=[1,2]).mean()
        
        log_gamma = torch.log(gamma + 1e-20)
        cluster_weights = torch.softmax(self.cluster_weights, dim=0)
        kl_c = torch.sum(gamma * (log_gamma - torch.log(cluster_weights.unsqueeze(0)) + 1e-20), dim=1).mean()
        
        kl_weight = min(1.0, current_epoch/150 * 1.0)
        total_loss = recon_loss + kl_weight * (kl_z + kl_c)
        
        return total_loss, recon_loss, kl_z, kl_c

    def contrastive_loss(self, z_projected, av_projected, cluster_labels, temperature=0.1):
        z_projected = z_projected / (z_projected.norm(dim=1, keepdim=True) + 1e-10)
        av_projected = av_projected / (av_projected.norm(dim=1, keepdim=True) + 1e-10)
        
        sim_matrix = torch.mm(z_projected, av_projected.t()) / temperature
        
        pos_mask = (cluster_labels.unsqueeze(1) == cluster_labels.unsqueeze(0)).float()
        pos_mask.fill_diagonal_(0)
        
        max_sim = sim_matrix.max(dim=1, keepdim=True)[0].detach()
        exp_sim = torch.exp(sim_matrix - max_sim)
        
        pos_term = (exp_sim * pos_mask).sum(dim=1)
        neg_term = (exp_sim * (1 - pos_mask)).sum(dim=1)
        
        loss = -(torch.log(pos_term + 1e-10) - torch.log(neg_term + 1e-10)).mean()
        return loss

    def predict_cluster(self, x):
        with torch.no_grad():
            _, _, z = self.encoder(x)
            soft_clusters = self.cluster_probabilities(z)
            return z, torch.argmax(soft_clusters, dim=1)

def embedding_extractor(device, shuffle=True):
    AV_scores = pd.read_csv(os.path.join(os.getcwd(), '..', 'CSV_files', 'arousal_valence.csv'))
    audios = {index:os.path.join(os.getcwd(), '..', 'audios', file) for index, file in enumerate(sorted(AV_scores['file'].unique()))}

    if os.path.exists('audio_embeddings.pt'):
        embeddings = torch.load('audio_embeddings.pt', map_location='cpu')
        frames_per_audio = int(embeddings.shape[0] / len(audios))
        ids = [id for id in audios for _ in range(frames_per_audio)]

    else:
        vggish_processer, vggish_model = VGGISH.get_input_processor(), VGGISH.get_model().to(device)
        vggish_model.eval()

        ids, vgg_, openl3_ = [], [], []

        for id, path in tqdm(audios.items(), desc="Extracting audio embeddings..."):

            vgg_data, vgg_sr = torchaudio.load(path)
            if vgg_sr != 16000: vgg_data = torchaudio.functional.resample(vgg_data, vgg_sr, 16000)
            if vgg_data.shape[0] > 1: vgg_data = vgg_data.mean(dim=0, keepdim=True)
            vgg_data = vgg_data.flatten()
            with torch.no_grad():
                vggish_embeddings = vggish_model(vggish_processer(vgg_data))

            openl3_data, openl3_sr = torchaudio.load(path)
            if openl3_data.shape[0] > 1: openl3_data.mean(dim=0, keepdim=True)
            openl3_data = openl3_data.flatten().numpy()
            target_hop_size = (len(openl3_data) / openl3_sr) / vggish_embeddings.shape[0]
            openl3_embeddings, _ = openl3.get_audio_embedding(openl3_data, openl3_sr, content_type="music", embedding_size=512, hop_size=target_hop_size, center=False)
            
            min_length = min(vggish_embeddings.shape[0], openl3_embeddings.shape[0])
            vggish_embeddings = vggish_embeddings[:min_length]
            openl3_embeddings = openl3_embeddings[:min_length]

            for _ in range(min_length): ids.append(id)
            vgg_.append(vggish_embeddings), openl3_.append(openl3_embeddings)

        vgg_, openl3_ = torch.cat(vgg_, dim=0), torch.from_numpy(np.concatenate(openl3_, axis=0))
        embeddings = torch.cat((vgg_, openl3_), dim=1)
        torch.save(embeddings, 'audio_embeddings.pt')

    AV_scores_filtered = AV_scores.drop(index=AV_scores[(AV_scores['valence'] == 0.0) & (AV_scores['arousal'] == 0.0)].index)
    AV_scores_filtered = AV_scores_filtered[['file','arousal','valence']].groupby(['file']).median()

    AV = []
    for id in ids:
        filename = os.path.basename(audios[id])
        AV.append(AV_scores_filtered.loc[AV_scores_filtered.index == filename].values)
    AV = torch.tensor(np.concatenate(AV))

    embeddings, AV = torch.from_numpy(MinMaxScaler().fit_transform(embeddings.numpy())).float().to(device), torch.from_numpy(MinMaxScaler(feature_range=(-1, 1)).fit_transform(AV.numpy())).float().to(device)

    dataset = TensorDataset(embeddings, AV)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=shuffle)

    return AV_scores, audios, ids, embeddings, dataloader

def train_model(embs, dataloader, hidden_dims, latent_dim, n_clusters, lr, epochs, device, display_clusters=False):
    model = VaDE(embs.shape[1], hidden_dims=hidden_dims, latent_dim=latent_dim, n_clusters=n_clusters).to(device)
    with torch.no_grad():
        _, _, initial_z = model.encoder(embs)
        kmeans = KMeans(n_clusters=n_clusters, random_state=77).fit(initial_z.cpu().numpy())
        model.cluster_means.data = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32).to(device)

    optimiser = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimiser, max_lr=lr*10, epochs=epochs, steps_per_epoch=len(dataloader), pct_start=0.3)

    train_RL, train_KLz, train_KLc, train_VL, train_CL, train_TL = [], [], [], [], [], []

    for epoch in range(epochs):
        model.train()
        epoch_RL, epoch_KLz, epoch_KLc, epoch_VL, epoch_CL, epoch_TL = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        epoch_clusters = []

        for batch in dataloader:
            X, av = batch[0], batch[1]

            mu, logvar, z, recon_X, av_encoded = model(X, av)
            gamma = model.cluster_probabilities(z)
            VL, RL, KL_z, KL_c = model.loss(X, recon_X, mu, logvar, gamma, epoch)
            
            cluster_labels = torch.argmax(gamma, dim=1)
            audio_projections = model.projector(z)
            av_projections = model.projector(av_encoded)
            CL = model.contrastive_loss(audio_projections, av_projections, cluster_labels)

            TL = VL + CL

            optimiser.zero_grad()
            TL.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
            optimiser.step()

            epoch_RL += RL.item()
            epoch_KLz += KL_z.item()
            epoch_KLc += KL_c.item()
            epoch_VL += VL.item()
            epoch_CL += CL.item()
            epoch_TL += TL.item()
            epoch_clusters.append(cluster_labels)

        epoch_RL /= len(dataloader)
        epoch_KLz /= len(dataloader)
        epoch_KLc /= len(dataloader)
        epoch_VL /= len(dataloader)
        epoch_CL /= len(dataloader)
        epoch_TL /= len(dataloader)

        train_RL.append(epoch_RL), train_KLz.append(epoch_KLz), train_KLc.append(epoch_KLc), train_VL.append(epoch_VL), train_CL.append(epoch_CL), train_TL.append(epoch_TL)
        scheduler.step()

        print(f'Epoch {epoch + 1}/{epochs}\nReconstruction Loss: {epoch_RL:.1f}. Cluster Divergence: {epoch_KLc:.1f}. Distribution Divergence: {epoch_KLz:.1f}. VaDE Loss: {epoch_VL:.1f}\nContrastive Loss: {epoch_CL:.1f}\nTotal Loss: {epoch_TL:.1f}\n')

        if display_clusters:
            print(f"{torch.bincount(torch.cat(epoch_clusters))}\n")

    all_z, all_clusters = [], []
    model.eval()
    for batch in dataloader:
        X = batch[0].to(device)
        z, clusters = model.predict_cluster(X)
        all_z.append(z.cpu().numpy()), all_clusters.append(clusters.cpu().numpy())
    all_z, all_clusters = np.concatenate(all_z), np.concatenate(all_clusters)

    return model, all_z, all_clusters, train_RL, train_KLz, train_KLc, train_VL, train_CL, train_TL

def precompute_audio_projections(model, dataloader):
    database = {'projected_audio': [], 'cluster_ids': []}
    
    model.eval()
    with torch.no_grad():
        for embs, _ in dataloader:
            z, clusters = model.predict_cluster(embs)
            proj = model.projector(z)
            database['projected_audio'].append(proj), database['cluster_ids'].append(clusters)

    database['projected_audio'] = torch.cat(database['projected_audio'])
    database['cluster_ids'] = torch.cat(database['cluster_ids'])

    return database

def retrieve_k_closest(AV_query, database, ids, audio_dict, AV_scores, model, device):
    model.eval()
    with torch.no_grad():
        query_tensor = torch.tensor(AV_query, dtype=torch.float32).to(device).unsqueeze(0)
        query_projected = model.projector(model.av_encoder(query_tensor))

    query_projected = F.normalize(query_projected, p=2, dim=1)
    database['projected_audio'] = F.normalize(database['projected_audio'], p=2, dim=1)

    similarities = torch.mm(query_projected, database['projected_audio'].T)
    topk_similarity, topk_indices = torch.topk(similarities.squeeze(), k=20)
    topk_similarity = topk_similarity.tolist()
    topk_clusters = database['cluster_ids'][topk_indices].tolist()
    topk_ids = [ids[index] for index in topk_indices.tolist()]

    unique_ids, unique_similarities, unique_clusters = [], [], []
    for sim, id, cluster in zip(topk_similarity, topk_ids, topk_clusters):
        if id not in unique_ids:
            unique_ids.append(id), unique_similarities.append(sim), unique_clusters.append(cluster)
            if len(unique_ids) == 10:
                break

    unique_filenames = [os.path.basename(audio_dict[id]) for id in unique_ids]
    AV_scores_deduped = AV_scores.drop_duplicates(subset='file')[['file','title','composer','film','director','genre']]

    results_df = pd.DataFrame({'file':unique_filenames, 'similarity':unique_similarities, 'cluster_id':unique_clusters}).merge(AV_scores_deduped, how='left', on='file')
    return results_df

In [ ]:
AV_scores, audio_dict, ids, embs, dataloader = embedding_extractor(device)

hidden_dims = [512, 256, 128]
latent_dim = 32
n_clusters = 6
lr = 0.00001
epochs = 200

model, latent_space, clusters, train_RL, train_KLz, train_KLc, train_VL, train_CL, train_TL = train_model(  embs=embs, dataloader=dataloader, hidden_dims=hidden_dims, latent_dim=latent_dim,
                                                                                                            n_clusters=n_clusters, lr=lr, epochs=epochs, device=device, display_clusters=False)

tsne = TSNE(n_components=2, random_state=42)
z_reduced = tsne.fit_transform(latent_space)

fig, ax = plt.subplots(1, 2, figsize=(20, 8))
ax[0].scatter(z_reduced[:, 0], z_reduced[:, 1], c=clusters, cmap='tab10', alpha=0.7, s=10)
ax[0].set_title("t-SNE Visualisation of Latent Space")
ax[0].set_xlabel("t-SNE Dimension 1")
ax[0].set_ylabel("t-SNE Dimension 2")

epoch_range = range(1, epochs + 1)
ax[1].plot(epoch_range, train_RL, label='Reconstruction Loss')
ax[1].plot(epoch_range, train_KLz, label='Distribution Divergence')
ax[1].plot(epoch_range, train_KLc, label='Cluster Divergence')
ax[1].plot(epoch_range, train_VL, label='VaDE Loss')
ax[1].plot(epoch_range, train_CL, label='Contrastive Loss')
ax[1].plot(epoch_range, train_TL, label='Total Loss')
ax[1].legend()
ax[1].set_title('Loss Components over Training Loop')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')

plt.show()

In [10]:
# SS = []
# for k in range(2, 11):
#     model, latent_space, clusters, _, _, _, _, _, _ = train_model(embs=embs,
#                                                                   dataloader=dataloader,
#                                                                   hidden_dims=hidden_dims,
#                                                                   latent_dim=latent_dim,
#                                                                   n_clusters=k,
#                                                                   lr=lr,
#                                                                   epochs=epochs,
#                                                                   device=device,
#                                                                   display_clusters=False)

#     if len(np.unique(clusters)) > 1:
#         SS.append(silhouette_score(latent_space, clusters, metric='cosine'))
#     else:
#         SS.append(-1)

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(SS)
# ax.set_xlabel('Number of clusters')
# ax.set_xticklabels([f"{tick + 2}" for tick in ax.get_xticks()])
# ax.set_ylabel('Silhouette Score')
# plt.show()

In [ ]:
AV_scores, audio_dict, ids, embs, dataloader = embedding_extractor(device, shuffle=False)

database = precompute_audio_projections(model, dataloader)

test = retrieve_k_closest((-0.7, 0.8), database, ids, audio_dict, AV_scores, model, device)

with open('audio_projections.pkl','wb') as f:
    pickle.dump(database, f)

with open('audio_dict.pkl','wb') as f:
    pickle.dump(audio_dict, f)

with open('ids.pkl', 'wb') as f:
    pickle.dump(ids, f)

torch.save(model.state_dict(), 'model_weights.pt')

In [12]:
# TODO: Implement updated model for survey (and send out)
# TODO: Write standalone web app for model
# TODO: Diss writing + other deliverables
# TODO: Tidy up code and fully understand